<div class="alert alert-info">
<font size="5"><b>Комментарий ревьюера</b></font>

Привет Мадина! Меня зовут Марат, и я буду твоим ревьюером. Спешу сообщить что все ключевые этапы в работе выполнены,  с задачей тебе удалось справиться. По поводу обращения - в IT сфере принято общаться на «ты» :) Но, если привычней на «вы», дай знать. Как ревьюера моя задача помочь тебе в развитии, дав хорошие советы. Я внимательно посмотрю твой код, ознакомлюсь с твоими выводами и оставлю комментарии. Где то могу предложить небольшие исправление в коде, но ненавязчиво. Где потребуются уточнения, я оставлю много наводящих вопросов. Они помогут тебя с поиском верного решения.

Все мои комментарии размечены по цветам, для лучшего восприятия.
    
<div class="alert alert-success">Зеленым цветом и словом «Успех» отмечены особо удачные и элегантные решения, которыми ты можешь гордиться. </div>
        
<div class="alert alert-warning">Желтым и значком словом «Совет», помечены решения у которых есть альтернативные решения, более оптимальные. Ты можешь найти их сразу и доработать проект, или отложить это на потом, для будущих проектах. Проект будет принят и без их доработки. </div>
        
<div class="alert alert-danger"> Красным цветом и значком словом «Ошибка» помечу твои решения, на которые стоит обратить внимание прежде всего. После их доработки проект будет принят. </div>
        
Залог успеха - работа сообща, взаимное уважение и работа в диалоге. Поэтому, помечай свои ответные комментарии на мои реплики заметным цветом или курсивом, так мне будет легче их отслеживать. Пожалуйста, не изменяй и не удаляй мои комментарии. Все это поможет выполнить повторную проверку быстрей.

</div>

In [1]:
import pandas as pd
import numpy as np
import re
import nltk
import spacy
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline

from sklearn.tree import DecisionTreeClassifier
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score
from nltk.corpus import stopwords
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


# Описание проекта

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

## Подготовка

In [2]:
try:
  df = pd.read_csv('/content/toxic_comments.csv')
except:
  df = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
df.shape

(159292, 3)

In [5]:
df['toxic'].value_counts()

toxic
0    143106
1     16186
Name: count, dtype: int64

In [6]:
df.columns

Index(['Unnamed: 0', 'text', 'toxic'], dtype='object')

In [7]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)

<div class="alert alert-warning">
<font size="5"><b>Комментарий ревьюера</b></font>

Совет:


Если не знаешь - чтобы не было столбца  `Unnamed: 0` при чтении файла можно так:


    pd.read_csv(..., index_col=0)

    
(`Unnamed: 0` появляется при не совсем корректном сохранении файла)    


Unnamed: 0 это "след" старых индексов. Если ты уберёшь первые 10 примеров и своего датасета, сохранишь его, а потом откроешь,  то появится столбец Unnamed: 0 начиная с цифры 9, и появится новый индексы начиная с нуля


Но это мелочь,  даже не нужно ничего исправлять. Просто знай, чтобы увидев такое в чужом коде не удивляться что бы это могло означать

In [ ]:
df.isnull().sum()

text     0
toxic    0
dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


<div class="alert alert-danger">
<font size="5"><b>Комментарий ревьюера</b></font>

Ошибка:



Да, в тренажере был текст на кирилице, там перевод в unicode оправдан. В нашем случае (латиница) это лишь  увеличит количество потребляемой памяти и это в лучшем случаи, в худшем он обрушает ядро.





</div>

In [8]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
# Загрузка модели spaCy
nlp = spacy.load('en_core_web_sm')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
def lemmatized(text):
    text = re.sub(r'[^a-zA-Z ]', ' ', text)  # Удаление неалфавитных символов
    text = text.lower()  # Приведение к нижнему регистру
    token = nltk.word_tokenize(text)  # Токенизация текста
    text = [word for word in token if word not in stop_words]  # Удаление стоп-слов

    # Лемматизация с использованием spaCy
    doc = nlp(' '.join(text))
    text = [token.lemma_ for token in doc]

    text = ' '.join(text)
    return text

In [11]:
nltk.download('punkt')
nltk.download('wordnet')

sentence1 = "The striped bats are hanging on their feet for best"
sentence2 = "you should be ashamed of yourself went worked"
df_my = pd.DataFrame([sentence1, sentence2], columns = ['text'])
print(df_my)


print(df_my['text'].apply(lemmatized))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


                                                text
0  The striped bats are hanging on their feet for...
1      you should be ashamed of yourself went worked
0    stripe bat hang foot well
1              ashamed go work
Name: text, dtype: object


<div class="alert alert-success">
<font size="5"><b>Комментарий ревьюераV2</b></font>



Успех 👍:



👍 Работает


 

<div class="alert alert-danger">
<font size="5"><b>Комментарий ревьюера</b></font>

Ошибка:



- WordNetLemmatizer  рабочий вариант, но у него есть особенности, для корректной работы ему нужно передавать не просто слово, но и POS-тег (Part of Speech, части речи). Набираемся ума-разума [тут](https://webdevblog.ru/podhody-lemmatizacii-s-primerami-v-python/) )  Обрати внимание на функцию `get_wordnet_pos`. Сразу хочу предупредить, что если делать Лемматизацию правильно, сучетом постегов, то время может занять полчаса-час. Так что не удивляйся (вообще советую сохранить результаты Лематизации в каком-то файлике, чтобы каждый раз не тратить на это много времени)
  

<div class="alert alert-warning">


Совет:




- лемматизацию можно было сделать с помощью SpaCy лемматизатором и прямо скажем как инструмент он более удобен и универсален, не нужно заморачиваться с токенизацией и учётом пос тегов




- Чтобы сэкономить время, и убедиться что всё отработало нормально, берёшь парочку предложений, создаёшь dataframe
    
    
    sentence1 = "The striped bats are hanging on their feet for best"
    sentence2 = "you should be ashamed of yourself went worked"
    df_my = pd.DataFrame([sentence1, sentence2], columns = ['text'])
    print(df_my)


    print(df_my['text'].apply(func))
    
    
    
И тестируешь не нем, должно получиться
    
    
    
    striped  ------> strip, went -------> go  



Если всё получилось, то можно использовать на всём датасете

In [12]:
df['lemmatized'] = df['text'].apply(lemmatized)

In [13]:
df.head()

,text,toxic,lemmatized
0,Explanation\nWhy the edits made under my usern...,0,explanation edit make username hardcore metall...
1,D'aww! He matches this background colour I'm s...,0,aww match background colour seemingly stuck th...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man really try edit war guy constantly rem...
3,"""\nMore\nI can't make any real suggestions on ...",0,make real suggestion improvement wonder sectio...
4,"You, sir, are my hero. Any chance you remember...",0,sir hero chance remember page


<div class="alert alert-success">
<font size="5"><b>Комментарий ревьюера</b></font>

Успех:


- Плюс за использование apply, неэффективные циклы нам ни к чему.


- Да, всегда лучше проверить что получилось  в итоге, так всегда будет возможность поправить ошибку

<div class="alert alert-warning">


Совет:


    
- попробуй .progress_apply, делает что .apply, но еще и показывает на какой итерации находится процесс.  

Для некоторых версий, чтобы заработал .progress_apply предварительно нужно сделать:
    
    
    from tqdm.notebook import tqdm
    tqdm.pandas()
    

И cудя по всему импорты нужно засунуть внутрь функции

То же самое делает .swifter.apply  Предварительно


    !pip install swifter
    import swifter



- если  процесс лемматизации затягивается, можно попробовать https://stackoverflow.com/questions/50992974/nltk-wordnetlemmatizer-not-lemmatizing-as-expected




- Можно построить [облако слов](https://habr.com/ru/post/517410/) - чтобы получить общее представление о тематике и о наиболее часто встречаемых словах в токсичных и нетоксичных твитах Кроме того графики, рисунки делают проект визуально интересней
    
В тренажере облако импортируем так

    !/opt/conda/bin/python -m pip install wordcloud


или

    !/opt/conda/bin/python -m pip install wordcloud==1.8.2.2  


И возможно дополнительно надо будет сделать



    !pip install --upgrade Pillow  (попробуй версию 9.5.0)



## Обучение

In [14]:
data = df.copy()
y = data['toxic']
data = data.drop(['toxic'], axis = 1)

In [15]:
x_train, x_test, y_train, y_test = train_test_split(data, y, test_size = 0.3, random_state = 42, stratify = y)

<div class="alert alert-danger">
<font size="5"><b>Комментарий ревьюера</b></font>

Ошибка:


Не забываем при   о random_state, иначе после каждого запуска кода у нас может быть разный сплит и сдедовательно результат


</div>


In [16]:
print('Train data shape:', x_train.shape, y_train.shape)
print('Test data shape:', x_test.shape, y_test.shape)

Train data shape: (111504, 2) (111504,)
Test data shape: (47788, 2) (47788,)


<div class="alert alert-success">
<font size="5"><b>Комментарий ревьюера</b></font>

Успех:



- правильно разбито на 2 выборки (иногда студенты использующие GS разбивают на 3 датасета)


- здорово что используешь stratify    



    
- плюс за  проверку
    
    



In [17]:
def training(model, params):
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(min_df = 1)),
        ('model', model)])
    grid = GridSearchCV(pipeline, cv = 5, n_jobs = -1, param_grid = params ,scoring = 'f1', verbose = False)
    grid.fit(x_train['lemmatized'], y_train)
    print('Лучший результат:', grid.best_score_)
    print('Лучшие параметры:', grid.best_params_)
    return grid

In [24]:
lr_mod = training(LogisticRegression(random_state=42), {"model__C":[10.1, 10.5, 15.0, 15.5], "model__penalty":["l2"]})

Лучший результат: 0.7755593115407349
Лучшие параметры: {'model__C': 15.0, 'model__penalty': 'l2'}


<div class="alert alert-danger">
<font size="5"><b>Комментарий ревьюера</b></font>

Ошибка:


Не забываем при инициализации модели о random_state

In [19]:
dtc_mod = training(DecisionTreeClassifier(random_state=42), {'model__criterion':['gini','entropy'],'model__max_depth':[2,4,6]})

Лучший результат: 0.5383984511535189
Лучшие параметры: {'model__criterion': 'gini', 'model__max_depth': 6}


In [21]:
nb_mod = training(MultinomialNB(), {
    "model__alpha": [0.1, 0.5, 1.0]
})

Лучший результат: 0.6316399572495219
Лучшие параметры: {'model__alpha': 0.1}


<div class="alert alert-success">
<font size="5"><b>Комментарий ревьюера</b></font>

Успех:






Корректно использован GridSearch + pipeline. Есть и другие варианты, тюнинга гиперпараметров, можешь [ознакомиться](https://www.freecodecamp.org/news/hyperparameter-optimization-techniques-machine-learning/). Выделю оptuna, очень много плюсов, причем изучение можно начать с [**OptunaSearchCV**](https://optuna.readthedocs.io/en/stable/reference/generated/optuna.integration.OptunaSearchCV.html). Интерфейс практически такой же, как у GridSearchCV, поэтому очень легко начать пользоваться



Самый лучший результат у Логистической регрессии где f1 = 0.77.

In [22]:
res = f1_score(y_test, lr_mod.predict(x_test['lemmatized']))

In [23]:
res

0.7751064074542735

<div class="alert alert-success">
<font size="5"><b>Комментарий ревьюера</b></font>

Успех:

- Все верно, логика моделирования не нарушена, тут тестируем только лучшую модель отобранную на валидации, или парочку лучших, если на валидации результаты близки




- Если студент получил на тесте f1 выше 0,75, это считается приемлемым результатом.


<div class="alert alert-warning">



Совет:


Что может помочь добиться лучшего результата (от простого)?


- можно поиграться [порогом](https://machinelearningmastery.com/threshold-moving-for-imbalanced-classification/). Таким образом можно поднять метрику на процент - полтора
    
     

- сгенерировать новые фичи, например  например посчитать число слов в тексте, длину слов итп итд. Или с помощью [тематического моделирования](https://pythobyte.com/python-for-nlp-topic-modeling-8fb3d689/)
    
    
    


    
- попробовать другие модели. проект своеобразный выбор между вычислительными ограничениями (много примеров, расчеты могут затянуться) и задачей получить хорошую метрику. С этой точки зрения  интересная [моделька](https://medium.com/geekculture/passive-aggressive-algorithm-for-big-data-models-8cd535ceb2e6) (открывается с помощью VPN) [или](https://datafinder.ru/products/passivno-agressivnyy-klassifikator-v-mashinnom-obuchenii). Она считается очень шустрой     
    


- использование предбученной модели Берта, выбрав соответствующую модель и используя полученные эмбединги, даже на небольшом тренировочном датасете можно обучить модель, которая на test покажет хорошую метрику. В этом случаи можно сразу получить метрику > 0.95 (при правильно выбранной модели)







<div class="alert alert-warning">
<font size="5"><b>Комментарий ревьюера</b></font>


Совет:


   


Можно оценить важность признаков, это можно сделать с помощью [.feature_importance](https://inria.github.io/scikit-learn-mooc/python_scripts/dev_features_importance.html), [Permutation Importance](https://scikit-learn.org/stable/modules/permutation_importance.html), shap
    
    
    
Полезно чтобы модель не выглядела чёрным ящиком   

## Выводы

Для задачи классификации комментариев в интернет-магазине «Викишоп» была разработана модель для выявления токсичных комментариев. Целью проекта было создание инструмента, который бы автоматически отправлял токсичные комментарии на модерацию. В ходе исследования были обучены и оценены три модели:

**Логистическая регрессия (Logistic Regression):**

- Лучший результат (F1-метрика на валидационном наборе данных): 0.7754713944326117
- Результат на тестовом наборе данных: 0.7751064074542735

**Дерево решений (DecisionTreeClassifier):**
- Результат (F1-метрика): 0.53

**Наивный байесовский классификатор (MultinomialNB):**

- Результат (F1-метрика): 0.63

На основе результатов можно сделать вывод, что наилучшие показатели продемонстрировала модель логистической регрессии с F1-метрикой 0.775 на валидационном наборе данных и 0.775 на тестовом наборе данных, что превышает требуемый порог в 0.75. Таким образом, эта модель рекомендуется к использованию для задачи классификации комментариев на позитивные и негативные в интернет-магазине «Викишоп».


<div class="alert alert-info">
<font size="5"><b>Комментарий ревьюера</b></font>



Мадина, у тебя старательно выполненная работа, все четко, осмысленно.



Нет проблем с комментированием кода - всё что ты делаешь понятно



Очистка проведена корректно (но есть вопросы к лемматизации)


Выводы присутствуют, они четкие и подробные.


Логика моделирования не нарушена, GS использован корректно. И здорово что он использован  вместе с pipeline


Так как число ошибок небольшое, можешь усложнить проект:







- попробуй очень современный и модный сейчас подход с использованием  Берта. Есть несколько вариантов, самый эффективный - это использовать эмбединги (как замена TFIDF) для этого есть уже готовый код в тренажёре (в этом случае разрешается сильно порезать датасет, а если еще и использовать GPU в Colab код можно прогнать за полчаса).



- для красивой  визуализации можешь построить облако для токсичных и нетоксичных комментариев



- чтобы заглянуть внутрь модели, можешь посмотреть какой из признаков является наиболее важный для логистической регресси (должно получиться слово fuck)   






Обязательное к исправлению:



- WordNetLemmatizer используем с POS - тег


- .astype('U') лишнее, стоит экономить ресурсы, иначе может даже ядро обрушиться




-  не забываем random_state (можно один раз вначале просто прописать random.seed(42), чтобы не прописывать каждый раз везде random_state. Кстати знаешь откуда 42?)





Жду исправлений, для принятия проекта. Если какие то вопросы, то сразу спрашивай )


<font color='green'><b>Полезные (и просто интересные) материалы:</b> \
Для работы с текстами используют и другие подходы. Например, сейчас активно используются RNN (LSTM) и трансформеры (BERT и другие с улицы Сезам, например, ELMO). НО! Они не являются панацеей, не всегда они нужны, так как и TF-IDF или Word2Vec + модели из классического ML тоже могут справляться. \
BERT тяжелый, существует много его вариаций для разных задач, есть готовые модели, есть надстройки над библиотекой transformers. Если, обучать BERT на GPU (можно в Google Colab или Kaggle), то должно быть побыстрее.\
https://huggingface.co/transformers/model_doc/bert.html \
https://t.me/renat_alimbekov \
https://colah.github.io/posts/2015-08-Understanding-LSTMs/ - Про LSTM \
https://web.stanford.edu/~jurafsky/slp3/10.pdf - про энкодер-декодер модели, этеншены\
https://pytorch.org/tutorials/beginner/transformer_tutorial.html - официальный гайд
по трансформеру от создателей pytorch\
https://transformer.huggingface.co/ - поболтать с трансформером \
Библиотеки: allennlp, fairseq, transformers, tensorflow-text — множествореализованных
методов для трансформеров методов NLP \
Word2Vec https://radimrehurek.com/gensim/models/word2vec.html


Если понравилась работа с текстами, то можешь посмотреть очень интересный (но очень-очень сложный) курс лекций: https://github.com/yandexdataschool/nlp_course .

Если нравится смотреть и слушать то есть целый курс на Ютубе https://www.youtube.com/watch?v=qDMwIQRQt-M&list=PLEwK9wdS5g0qksxWxtE5c2KuFkIfUXe3i&index=1


 
 

<div class="alert alert-info">
<font size="5"><b>Комментарий ревьюераV2</b></font>



Мадина, тобой исправлено красное, желтое не тронуто - твое право ) Тем не менее надеюсь мои советы и вопросики были полезны и ты узнала что то новое.
Отличная работа. Желаю успехов в дальнейшей учебе!
   